In [26]:
render_dict = ['.', 'P', 'T', '#', 'X']
def render_maze(maze: list[list[float]]) -> list[str]:
    ret = []
    for x in maze:
        row = ""
        for y in x:
            row += render_dict[y]
        ret.append(row)
    return ret

In [56]:
%load_ext autoreload
%autoreload 2

from util.maze_generator import maze_generator
from util.a_star_search import a_star_pathfinding
from util.a_star_search import _find_item

maze = maze_generator((8, 14))

path = a_star_pathfinding(maze)
startx, starty = _find_item(1, maze)
endx, endy = _find_item(2, maze)
for x, y in path:
    if x == startx and y == starty:
        continue
    if x == endx and y == endy:
        continue
    maze[x][y] = 4

render_maze(maze)

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload
[[None, None, None, (1, 4), (1, 5), (0, 6), (1, 7), (0, 6), None, None, None, None, None, None], [None, None, (1, 3), (0, 3), (0, 5), (1, 6), (0, 7), (2, 8), (0, 7), (2, 9), None, None, None, None], [None, None, None, None, None, None, (1, 7), (3, 7), (3, 8), (2, 8), None, None, None, None], [None, None, None, None, None, (3, 6), (2, 6), (3, 6), (4, 9), (2, 9), None, None, None, (4, 13)], [None, None, None, None, None, None, None, None, (5, 9), (5, 10), None, (4, 12), (5, 13), (5, 12)], [None, None, None, None, None, None, None, None, None, (6, 9), (4, 11), (5, 12), (4, 11), None], [None, None, None, None, None, None, None, None, (5, 9), (5, 10), (6, 11), (5, 11), (6, 13), (5, 12)], [None, None, None, None, None, None, None, None, None, None, None, None, None, None]]


['###...T.######',
 '##.....X..####',
 '######..X.####',
 '#####...X.###.',
 '########.X#XX.',
 '#########.X..P',
 '########......',
 '##############']

In [40]:
t = [[None, None, None, None, None, None, None, (0, 6), (0, 7), None, None, None, None, None], 
[None, None, None, None, None, None, (0, 7), (1, 6), (1, 7), (1, 8), (1, 9), (1, 10), (2, 13), (2, 12)], 
[None, None, None, (3, 3), (3, 5), None, (1, 7), (2, 6), (2, 7), (2, 8), None, None, (1, 11), (1, 13)], 
[None, None, None, (2, 4), (2, 3), (2, 6), (3, 5), (3, 6), (3, 7), (3, 8), (3, 9), None, (2, 13), (3, 12)], 
[None, (4, 2), (3, 3), (4, 2), (4, 3), None, None, (3, 8), None, (3, 10), None, (3, 10), (4, 11), (4, 12)], 
[(4, 1), (5, 0), None, (4, 4), None, None, None, None, None, None, None, None, (4, 13), (5, 12)], 
[(5, 1), (6, 0), None, None, None, None, None, None, None, None, None, None, None, None], 
[(6, 1), None, None, None, None, None, None, None, None, None, None, None, None, None]]

i = 0
for x in t:
    s = ""
    for y in x: 
        if y is None:
            s += "( ,   )"
        else: 
            s += f"{str(y):<{7}}"
    print(s)

#    012345678
# 0['######XX.#####',
# 1 '######XX......',
# 2 '###..#X...##..',
# 3 '###..T.....#..',
# 4 '#....##P#.#...',
# 5 '..#.########..',
# 6 '..############',
# 7 '.#############']

#        0      1      2      3      4      5      6     7      8       9
# 0 ( ,   )( ,   )( ,   )( ,   )( ,   )( ,   )( ,   )(0, 6) (0, 7) ( ,   )( ,   )( ,   )( ,   )( ,   )
# 1 ( ,   )( ,   )( ,   )( ,   )( ,   )( ,   )(0, 7) (1, 6) (1, 7) (1, 8) (1, 9) (1, 10)(2, 13)(2, 12)
# 2 ( ,   )( ,   )( ,   )(3, 3) (3, 5) ( ,   )(1, 7) (2, 6) (2, 7) (2, 8) ( ,   )( ,   )(1, 11)(1, 13)
# 3 ( ,   )( ,   )( ,   )(2, 4) (2, 3) (2, 6) (3, 5) (3, 6) (3, 7) (3, 8) (3, 9) ( ,   )(2, 13)(3, 12)
# 4 ( ,   )(4, 2) (3, 3) (4, 2) (4, 3) ( ,   )( ,   )(3, 8) ( ,   )(3, 10)( ,   )(3, 10)(4, 11)(4, 12)
# 5 (4, 1) (5, 0) ( ,   )(4, 4) ( ,   )( ,   )( ,   )( ,   )( ,   )( ,   )( ,   )( ,   )(4, 13)(5, 12)
# 6 (5, 1) (6, 0) ( ,   )( ,   )( ,   )( ,   )( ,   )( ,   )( ,   )( ,   )( ,   )( ,   )( ,   )( ,   )
# 7 (6, 1) ( ,   )( ,   )( ,   )( ,   )( ,   )( ,   )( ,   )( ,   )( ,   )( ,   )( ,   )( ,   )( ,   )

( ,   )( ,   )( ,   )( ,   )( ,   )( ,   )( ,   )(0, 6) (0, 7) ( ,   )( ,   )( ,   )( ,   )( ,   )
( ,   )( ,   )( ,   )( ,   )( ,   )( ,   )(0, 7) (1, 6) (1, 7) (1, 8) (1, 9) (1, 10)(2, 13)(2, 12)
( ,   )( ,   )( ,   )(3, 3) (3, 5) ( ,   )(1, 7) (2, 6) (2, 7) (2, 8) ( ,   )( ,   )(1, 11)(1, 13)
( ,   )( ,   )( ,   )(2, 4) (2, 3) (2, 6) (3, 5) (3, 6) (3, 7) (3, 8) (3, 9) ( ,   )(2, 13)(3, 12)
( ,   )(4, 2) (3, 3) (4, 2) (4, 3) ( ,   )( ,   )(3, 8) ( ,   )(3, 10)( ,   )(3, 10)(4, 11)(4, 12)
(4, 1) (5, 0) ( ,   )(4, 4) ( ,   )( ,   )( ,   )( ,   )( ,   )( ,   )( ,   )( ,   )(4, 13)(5, 12)
(5, 1) (6, 0) ( ,   )( ,   )( ,   )( ,   )( ,   )( ,   )( ,   )( ,   )( ,   )( ,   )( ,   )( ,   )
(6, 1) ( ,   )( ,   )( ,   )( ,   )( ,   )( ,   )( ,   )( ,   )( ,   )( ,   )( ,   )( ,   )( ,   )


In [45]:
ENV_WALLS = [
    [0, 0, 0, 3, 0, 2, 0, 0, 0],
    [0, 1, 0, 3, 0, 0, 0, 0, 0],
    [0, 0, 0, 3, 3, 3, 3, 0, 0],
    [0, 0, 0, 3, 0, 0, 0, 0, 0],
    [0, 0, 0, 0, 0, 3, 3, 3, 0],
]

path = a_star_pathfinding(ENV_WALLS)
startx, starty = _find_item(1, ENV_WALLS)
endx, endy = _find_item(2, ENV_WALLS)
for x, y in path:
    if x == startx and y == starty:
        continue
    if x == endx and y == endy:
        continue
    ENV_WALLS[x][y] = 4

render_maze(ENV_WALLS)

[[None, (0, 0), (0, 1), None, (1, 4), (1, 6), (1, 5), (1, 8), (1, 7)], [(0, 1), (1, 0), (1, 1), None, (0, 5), (0, 4), (2, 7), (0, 6), (0, 8)], [(1, 1), (1, 2), (2, 1), None, None, None, None, (3, 6), (1, 8)], [(2, 1), (2, 2), (3, 1), None, (4, 3), (3, 4), (3, 5), (2, 8), (3, 7)], [(3, 1), (4, 0), (4, 1), (4, 2), (4, 3), None, None, None, (3, 7)]]


['...#.T...', '.PX#..X..', '.XX####X.', '.X.#XXX..', 'XXXX.###.']